# Functions and imports

In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re#
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python/INPUT_FILE.csv --types_file data_python/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
         --miss_percentage_train 0 --miss_percentage_test 0 \
        --true_miss_file data_python/MISS_FILE --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
# get file list
files=[i for i in os.listdir('data_python/') if not '_type' in i and not '_missing' in i and i not in '.DS_Store']
print(files)
files.sort()
print(len(files))

['MMSE_Language_VIS36.csv', 'MMSE_Working_Memory_Registration_VIS36.csv', 'volume_VIS1.csv', 'Inflammatory_response_subgraph_VIS1.csv', 'GSK3_subgraph_VIS1.csv', 'ADAM_Metallopeptidase_subgraph_VIS1.csv', 'Tau_protein_subgraph_VIS1.csv', 'MMSE_Attention_Concentration_VIS12.csv', 'volume_VIS12.csv', 'Interferon_signaling_subgraph_VIS1.csv', 'MMSE_Language_VIS24.csv', 'MMSE_Working_Memory_Registration_VIS24.csv', 'volume_VIS6.csv', 'MMSE_Orientation_VIS12.csv', 'MMSE_Orientation_VIS6.csv', 'Insulin_signal_transduction_VIS1.csv', 'Lipid_metabolism_subgraph_VIS1.csv', 'Cholesterol_metabolism_subgraph_VIS1.csv', 'MMSE_Memory_Recall_VIS24.csv', 'Nerve_growth_factor_subgraph_VIS1.csv', 'MMSE_Memory_Recall_VIS36.csv', 'Caspase_subgraph_VIS1.csv', 'ATP_binding_cassette_transport_subgraph_VIS1.csv', 'MMSE_Attention_Concentration_VIS6.csv', 'imagingPET_VIS1.csv', 'MMSE_Memory_Recall_VIS12.csv', 'Matrix_metalloproteinase_subgraph_VIS1.csv', 'MMSE_Memory_Recall_VIS6.csv', 'MMSE_Language_VIS6.csv', 

In [3]:
file = pd.read_csv('data_python/' + files[1], header=None)
sample_size=len(file)

In [4]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/HI-VAE'

# General settings

In [5]:
best_hyper = pd.read_csv('best_hyper_ADNI_processed_upd_rename.csv', sep = ',')

In [6]:
best_hyper

,Unnamed: 0,act,loss,lrates,nbatch,wdecay,ydims,files,sdims
0,1,relu,2.517873,0.010,32,0.000,1,ADAM_Metallopeptidase_subgraph_VIS1.csv,3
1,2,relu,0.994835,0.010,32,0.000,1,APOE_subgraph_VIS1.csv,2
2,3,relu,20.522337,0.010,16,0.000,1,ATP_binding_cassette_transport_subgraph_VIS1.csv,4
3,4,none,2.171016,0.010,16,0.000,1,Amyloidogenic_subgraph_VIS1.csv,2
4,5,tanh,5.066627,0.010,32,0.000,1,Apoptosis_signaling_subgraph_VIS1.csv,3
5,6,tanh,2.004648,0.010,16,0.000,1,Axonal_guidance_subgraph_VIS1.csv,2
6,7,tanh,5.903451,0.010,16,0.001,1,Caspase_subgraph_VIS1.csv,3
7,8,tanh,0.204889,0.010,16,0.000,1,Chemokine_signaling_subgraph_VIS1.csv,3
8,9,none,3.398716,0.010,32,0.000,1,Cholesterol_metabolism_subgraph_VIS1.csv,3
9,10,none,0.734978,0.010,16,0.000,1,GSK3_subgraph_VIS1.csv,3


# VP decoding

Run after bnet.R

In [7]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/HI-VAE'

In [8]:
meta = pd.read_csv('metaenc.csv')
meta


,scode_ADAM_Metallopeptidase_subgraph,SUBJID,zcode_ADAM_Metallopeptidase_subgraph,scode_APOE_subgraph,zcode_APOE_subgraph,scode_ATP_binding_cassette_transport_subgraph,zcode_ATP_binding_cassette_transport_subgraph,scode_Amyloidogenic_subgraph,zcode_Amyloidogenic_subgraph,scode_Apoptosis_signaling_subgraph,...,scode_imagingPET_VIS1,zcode_imagingPET_VIS1,scode_volume_VIS1,zcode_volume_VIS1,scode_volume_VIS12,zcode_volume_VIS12,scode_volume_VIS24,zcode_volume_VIS24,scode_volume_VIS6,zcode_volume_VIS6
0,1,002_S_0295,1.793218,0,8.089138,0,-0.848805,1,-4.383145,0,...,0,0.571260,0,-0.443032,0,0.443607,0,-0.550346,0,0.329455
1,0,002_S_0413,-3.791347,1,-7.432476,0,-2.570424,1,-5.411586,0,...,0,0.571260,0,-0.019549,0,1.198814,0,-0.229604,0,-0.017671
2,1,002_S_0559,1.690306,1,-6.022939,1,-7.816886,1,-5.421945,2,...,0,0.571260,0,-0.300117,0,0.608391,0,-0.204198,0,0.252057
3,0,002_S_0619,-3.916058,0,7.978750,0,-1.385342,0,9.970449,0,...,0,0.571260,0,-2.935828,0,-0.918872,0,-1.945647,0,1.607824
4,1,002_S_0685,1.855543,1,-6.022939,0,-1.153458,1,-4.383145,2,...,0,0.571260,0,-0.473426,0,0.366722,0,-0.623609,0,0.205199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,0,941_S_4377,-3.729022,1,-7.380951,1,-7.867011,0,9.984890,2,...,0,-0.298578,0,0.012542,0,0.846858,0,-0.310240,0,0.016990
1441,0,941_S_4420,-3.637777,0,7.977829,3,-1.573903,1,-4.795940,1,...,0,0.639755,0,-0.083528,0,0.581044,0,-0.310240,0,0.318404
1442,0,941_S_4764,-3.637777,0,7.977829,3,-1.573903,1,-4.795940,1,...,0,0.177547,0,-0.651771,0,0.148012,0,-1.176463,0,0.252057
1443,0,941_S_5124,-3.637777,0,7.977829,3,-1.573903,1,-4.795940,1,...,0,0.992131,0,-0.494627,0,0.225685,0,-1.157630,0,0.252057


In [9]:
VPcodes  = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/data_out/main_VirtualPPts.csv')
VPcodes

,scode_ADAM_Metallopeptidase_subgraph_VIS1,zcode_ADAM_Metallopeptidase_subgraph_VIS1,scode_APOE_subgraph_VIS1,zcode_APOE_subgraph_VIS1,scode_ATP_binding_cassette_transport_subgraph_VIS1,zcode_ATP_binding_cassette_transport_subgraph_VIS1,scode_Amyloidogenic_subgraph_VIS1,zcode_Amyloidogenic_subgraph_VIS1,scode_Apoptosis_signaling_subgraph_VIS1,zcode_Apoptosis_signaling_subgraph_VIS1,...,SA_Flexibility_VIS36,SA_VisualPerception_VIS36,SA_Planning_VIS36,SA_SpatialMemory_VIS36,SA_ProspectiveMemory_VIS36,visitmiss_VIS6,visitmiss_VIS12,visitmiss_VIS18,visitmiss_VIS24,visitmiss_VIS36
0,0,-3.612736,1,-6.109492,1,-8.214150,1,-5.153852,0,-0.341706,...,32.741786,40.991553,58.476916,37.340559,50.668796,0,1,1,1,1
1,0,-3.621310,0,8.067755,3,-1.555055,1,-4.860696,1,-2.158001,...,-291.415018,15141.420491,-7067.546164,-3505.214069,-1595.725863,0,0,1,0,0
2,2,-16.317607,1,-6.137707,3,-1.667300,1,-5.292013,1,-2.158928,...,102.551169,-2220.169612,-126.042937,-51.687270,-7.088049,0,0,1,1,1
3,2,-16.418531,0,7.952840,0,1.600433,1,-4.711647,2,-8.617427,...,47.358982,44.775376,43.632494,38.483393,38.853013,0,1,1,0,0
4,2,-16.380413,0,8.096202,1,-8.910464,1,-4.803211,1,-2.166835,...,65.106790,34.642166,38.625096,31.020242,41.589969,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,2,-16.371486,0,8.064978,0,0.960880,1,-4.592659,1,-2.146936,...,39.339881,42.995184,46.786336,46.332733,51.906586,0,0,1,1,1
1441,2,-16.322898,0,7.921854,1,-7.298797,1,-4.920839,2,-8.620975,...,34.634688,35.695008,25.315827,33.628011,43.382722,0,0,1,1,1
1442,0,-3.743484,1,-5.668591,0,1.881876,0,9.967143,0,-0.377902,...,43.708840,37.766068,40.614891,31.131820,32.164525,0,0,1,0,0
1443,2,-16.592008,0,8.019706,3,-1.628679,1,-4.189525,1,-2.108793,...,61.141916,44.225081,44.451471,44.671583,36.800241,0,0,1,0,1


In [10]:
VPcodes.columns

Index(['scode_ADAM_Metallopeptidase_subgraph_VIS1',
       'zcode_ADAM_Metallopeptidase_subgraph_VIS1', 'scode_APOE_subgraph_VIS1',
       'zcode_APOE_subgraph_VIS1',
       'scode_ATP_binding_cassette_transport_subgraph_VIS1',
       'zcode_ATP_binding_cassette_transport_subgraph_VIS1',
       'scode_Amyloidogenic_subgraph_VIS1',
       'zcode_Amyloidogenic_subgraph_VIS1',
       'scode_Apoptosis_signaling_subgraph_VIS1',
       'zcode_Apoptosis_signaling_subgraph_VIS1',
       ...
       'SA_Flexibility_VIS36', 'SA_VisualPerception_VIS36',
       'SA_Planning_VIS36', 'SA_SpatialMemory_VIS36',
       'SA_ProspectiveMemory_VIS36', 'visitmiss_VIS6', 'visitmiss_VIS12',
       'visitmiss_VIS18', 'visitmiss_VIS24', 'visitmiss_VIS36'],
      dtype='object', length=316)

In [11]:
sample_size

1445

In [12]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/data_out/main_VirtualPPts.csv')

dfs=list()
virt=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    dec=helpers.dec_network(settings,zcodes,scodes,VP=True)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(dec)
    dat.columns=names
    dat['SUBJID']=subj
    virt.append(dec)
    dfs.append(dat)

virt_dic=dict(zip(files,virt))
decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP.csv',index=False)

VP
True
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders

[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.





/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Decoder...
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use `tf.random.categorical` instead.
[*] Defining Cost function...
Instructions for updating:

Future major versi

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/APOE_subgraph_VIS1/APOE_subgraph_VIS1.ckpt
Model restored: ./Saved_Networks/APOE_subgraph_VIS1/APOE_subgraph_VIS1.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 3) dtype=float32>: array([[1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]]), <tf.Tensor 'Placeholder_1:0' shape=(1445, 3) dtype=float32>: array([[1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])}
::::::::::::VP_misslist/APOE_subgraph_VIS1_vpmiss.csv
Number of cols
2
Number of cols
2
VP
True
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders


...

[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS12/volume_VIS12.ckpt
Model restored: ./Saved_Networks/volume_VIS12/volume_VIS12.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 1) dtype=float32>: array([[0.02455583],
       [0.01781828],
       [0.        ],
       ...,
       [0.04303788],
       [0.        ],
       [0.        ]]), <tf.Tensor 'Placeholder_1:0' shape=(1445, 1) dtype=float32>: array([[0.00417438],
       [0.00431979],
       [0.        ],
       ...,
       [0.00409647],
       [0.00305463],
       [0.        ]]), <tf.Tensor 'Placeholder_2:0' shape=(1445, 1) dtype=float32>: array([[0.65710239],
       [0.67277725],
       [0.        ],
       ...,
       [0.62222568],
       [0.        ],
       [0.        ]]), <tf.Tensor 'Placeholder_3:0' shape=(1445, 1) dtype=float32>: array([[0.00216288],
       [0.00271168],
       [0.        ],
       ...,
       [0.00259299],
       [0.       

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS24/volume_VIS24.ckpt
Model restored: ./Saved_Networks/volume_VIS24/volume_VIS24.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 1) dtype=float32>: array([[0.0257028 ],
       [0.01975326],
       [0.02312395],
       ...,
       [0.04595672],
       [0.        ],
       [0.01736007]]), <tf.Tensor 'Placeholder_1:0' shape=(1445, 1) dtype=float32>: array([[0.00414354],
       [0.00415081],
       [0.00432725],
       ...,
       [0.00398239],
       [0.00284695],
       [0.00448053]]), <tf.Tensor 'Placeholder_2:0' shape=(1445, 1) dtype=float32>: array([[0.64675106],
       [0.64593611],
       [0.64593928],
       ...,
       [0.60056079],
       [0.62910738],
       [0.69132382]]), <tf.Tensor 'Placeholder_3:0' shape=(1445, 1) dtype=float32>: array([[0.00233563],
       [0.00240375],
       [0.00234878],
       ...,
       [0.        ],
       [0.0020746

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS6/volume_VIS6.ckpt
Model restored: ./Saved_Networks/volume_VIS6/volume_VIS6.ckpt
::::::DECODING:::::::::
feed Dictionary
{<tf.Tensor 'Placeholder:0' shape=(1445, 1) dtype=float32>: array([[0.02376521],
       [0.01757167],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]]), <tf.Tensor 'Placeholder_1:0' shape=(1445, 1) dtype=float32>: array([[0.00419132],
       [0.00428514],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]]), <tf.Tensor 'Placeholder_2:0' shape=(1445, 1) dtype=float32>: array([[0.66229519],
       [0.66227801],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]]), <tf.Tensor 'Placeholder_3:0' shape=(1445, 1) dtype=float32>: array([[0.00222573],
       [0.00258178],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],


Get Loglikelihoods for R plot!

In [13]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/ADNI_VAMBN_paper_final/data/data_out/main_VirtualPPts.csv')
dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP=True)
    loglik=np.nanmean(np.array(loglik).T,axis=1)
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(loglik)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks.csv',index=False)

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=1445, data_file='data_python/ADAM_Metallopeptidase_subgraph_VIS1.csv', dim_latent_s=3, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ADAM_Metallopeptidase_subgraph_VIS1', true_miss_file='data_python/ADAM_Metallopeptidase_subgraph_VIS1_missing.csv', types_file='data_python/ADAM_Metallopeptidase_subgraph_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ADAM_Metallopeptidase_subgraph_VIS1/ADAM_Metallopeptidase_subgraph_VIS1.ckpt
Model restored: ./Saved_Networks/ADAM_Metallopeptidase_subgraph_VIS1/ADAM_Metallopeptidase_subgraph_VIS1.ckpt
::::::DECODING:::::::::
::::::::::::V

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=1445, data_file='data_python/APOE_subgraph_VIS1.csv', dim_latent_s=2, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='APOE_subgraph_VIS1', true_miss_file='data_python/APOE_subgraph_VIS1_missing.csv', types_file='data_python/APOE_subgraph_VIS1_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/APOE_subgraph_VIS1/APOE_subgraph_VIS1.ckpt
Model restored: ./Saved_Networks/APOE_subgraph_VIS1/APOE_subgraph_VIS1.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/APOE_subgraph_VIS1_vpmiss.csv
Namespace(batch_size=1445, data_file='data_python/ATP_binding_cassette_transport_subgraph_VIS1.

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


...

Namespace(batch_size=1445, data_file='data_python/volume_VIS12.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='volume_VIS12', true_miss_file='data_python/volume_VIS12_missing.csv', types_file='data_python/volume_VIS12_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS12/volume_VIS12.ckpt
Model restored: ./Saved_Networks/volume_VIS12/volume_VIS12.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/volume_VIS12_vpmiss.csv
Namespace(batch_size=1445, data_file='data_python/volume_VIS24.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, di

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS24/volume_VIS24.ckpt
Model restored: ./Saved_Networks/volume_VIS24/volume_VIS24.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/volume_VIS24_vpmiss.csv
Namespace(batch_size=1445, data_file='data_python/volume_VIS6.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='volume_VIS6', true_miss_file='data_python/volume_VIS6_missing.csv', types_file='data_python/volume_VIS6_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/volume_VIS6/volume_VIS6.ckpt
Model restored: ./Saved_Networks/volume_VIS6/volume_VIS6.ckpt
::::::DECODING:::::::::
::::::::::::VP_misslist/volume_VIS6_vpmiss.csv
